In [ ]:
!pip install -q datasets transformers accelerate peft bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd

df = pd.read_csv("hf://datasets/BI55/MedText/medtext_2.csv")
print(df.columns)
print(df.iloc[0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Index(['Prompt', 'Completion'], dtype='object')
Prompt        A 50-year-old male presents with a history of ...
Completion    This patient's history of recurrent kidney sto...
Name: 0, dtype: object


In [ ]:
from datasets import Dataset

formatted_df = pd.DataFrame({
    "instruction": df["Prompt"],
    "input": "",
    "output": df["Completion"]
})

dataset = Dataset.from_pandas(formatted_df)
dataset = dataset.train_test_split(test_size=0.1)

dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 1270
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 142
    })
})

In [ ]:
from transformers import AutoTokenizer

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [ ]:
def format_and_tokenize(example):
    full_prompt = f"Instruction: {example['instruction']}\nResponse:{example['output']}"
    return tokenizer(full_prompt, truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(format_and_tokenize, batched=False)

tokenized_dataset["train"][0]

Map:   0%|          | 0/1270 [00:00<?, ? examples/s]

Map:   0%|          | 0/142 [00:00<?, ? examples/s]

{'instruction': 'A 32-year-old woman complains of persistent difficulties focusing at work, feeling restless, and often forgetting important deadlines. She also has trouble organizing tasks and prioritizing her duties. These issues have been causing significant stress and have affected her work performance. What could be causing these symptoms?',
 'input': '',
 'output': "Although ADHD is commonly diagnosed in childhood, it can persist into adulthood or be diagnosed later in life. This woman's reported symptoms of persistent inattention, restlessness, forgetfulness, and difficulty with organization are consistent with Adult Attention Deficit Hyperactivity Disorder. The next step is to conduct a comprehensive evaluation that includes obtaining a detailed history of symptoms and their impact on her life.",
 'input_ids': [1,
  2799,
  4080,
  29901,
  319,
  29871,
  29941,
  29906,
  29899,
  6360,
  29899,
  1025,
  6114,
  15313,
  1144,
  310,
  28152,
  23553,
  12789,
  4746,
  472,

In [ ]:
from transformers import AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./tinyllama-medtext-lora",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_steps=200,
    save_total_limit=2,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

/tmp/ipython-input-7-3420245632.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,1.574400
20,1.401400
30,1.274400
40,1.279100
50,1.250900
60,1.248000
70,1.227800
80,1.269500
90,1.243400
100,1.292900


TrainOutput(global_step=477, training_loss=1.20667600531748, metrics={'train_runtime': 715.2713, 'train_samples_per_second': 5.327, 'train_steps_per_second': 0.667, 'total_flos': 1.212144773234688e+16, 'train_loss': 1.20667600531748, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./tinyllama-medtext-lora")
tokenizer.save_pretrained("./tinyllama-medtext-lora")

('./tinyllama-medtext-lora/tokenizer_config.json',
 './tinyllama-medtext-lora/special_tokens_map.json',
 './tinyllama-medtext-lora/chat_template.jinja',
 './tinyllama-medtext-lora/tokenizer.model',
 './tinyllama-medtext-lora/added_tokens.json',
 './tinyllama-medtext-lora/tokenizer.json')

In [ ]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model="./tinyllama-medtext-lora",
    tokenizer="./tinyllama-medtext-lora",
    device=0
)

prompt = "Instruction: 65-year-old female presents with fatigue, weight loss, and elevated calcium levels.\nResponse:"

output = pipe(prompt, max_new_tokens=100, do_sample=True, temperature=0.7)[0]["generated_text"]

print(output)

Device set to use cuda:0


Instruction: 65-year-old female presents with fatigue, weight loss, and elevated calcium levels.
Response:A 65-year-old female with a history of hypertension, diabetes, and hyperlipidemia is at a high risk for developing osteoporosis due to her age and poor calcium intake. Treatment usually involves a high-calcium diet and supplements, along with calcium supplements. Medication may be necessary to prevent fractures. This patient should also be encouraged to maintain a healthy l


In [ ]:
!pip install -q evaluate bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.9 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model="./tinyllama-medtext-lora", tokenizer="./tinyllama-medtext-lora", device=0)

eval_inputs = dataset["test"]

eval_results = []

for example in eval_inputs:
    prompt = f"Instruction: {example['instruction']}\nResponse:"
    output = pipe(prompt, max_new_tokens=100, do_sample=False)[0]["generated_text"]

    predicted_response = output.split("Response:")[-1].strip()

    eval_results.append({
        "instruction": example["instruction"],
        "prediction": predicted_response,
        "reference": example["output"]
    })

eval_results[0]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are no

{'instruction': 'A 55-year-old woman presents with fatigue and joint pain. She denies any fever, rash, or weight changes. Her vital signs are normal and physical examination is unremarkable.',
 'prediction': "This patient's symptoms and physical examination findings suggest a viral infection, possibly influenza, given the seasonal onset of symptoms in the fall. However, the lack of fever, rash, or weight changes could suggest a different cause. Further testing, including a viral culture and serology, should be considered. If the results are positive, the patient should be treated with antiviral medication. If the results are negative, the patient should be",
 'reference': 'Fatigue and joint pain could be indicative of numerous conditions, including autoimmune diseases such as rheumatoid arthritis or lupus, fibromyalgia, or even a thyroid disorder. A more detailed history, physical examination focusing on the joints, and possibly further investigations like rheumatologic lab tests, thyr

In [ ]:
!pip install rouge_score
!pip install bert_score

from evaluate import load

predictions = [ex["prediction"] for ex in eval_results]
references = [ex["reference"] for ex in eval_results]

rouge = load("rouge")
rouge_result = rouge.compute(predictions=predictions, references=references)

bertscore = load("bertscore")
bertscore_result = bertscore.compute(predictions=predictions, references=references, lang="en")

print("ROUGE Score:", rouge_result)
print("BERT Score:", bertscore_result)

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=3cfba17e1c5ffc09ba960eb32b5debd1633aa226c4fa9f0917a991086515ce53
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ROUGE Score: {'rouge1': np.float64(0.34929760740661675), 'rouge2': np.float64(0.10169899222809811), 'rougeL': np.float64(0.22688898822602732), 'rougeLsum': np.float64(0.22753795152941203)}
BERT Score: {'precision': [0.8523076772689819, 0.8983393907546997, 0.8650028705596924, 0.8849427700042725, 0.8615153431892395, 0.8980950117111206, 0.8903180956840515, 0.8667439222335815, 0.8877160549163818, 0.867202639579773, 0.8859006762504578, 0.8774937391281128, 0.868293046951294, 0.8739939332008362, 0.8826137781143188, 0.8858385682106018, 0.8837525844573975, 0.8348096609115601, 0.8824940919876099, 0.9015179872512817, 0.8902233839035034, 0.8855346441268921, 0.8573658466339111, 0.9039982557296753, 0.8782577514648438, 0.8859895467758179, 0.8679860830307007, 0.8753554821014404, 0.8884625434875488, 0.8657180666923523, 0.8820977210998535, 0.8613486289978027, 0.8843372464179993, 0.8749018907546997, 0.8784407377243042, 0.865119457244873, 0.8545421361923218, 0.8748545050621033, 0.9090421199798584, 0.88185

In [ ]:
from google.colab import files
model.save_pretrained("tinyllama-medtext-lora")
tokenizer.save_pretrained("tinyllama-medtext-lora")

('tinyllama-medtext-lora/tokenizer_config.json',
 'tinyllama-medtext-lora/special_tokens_map.json',
 'tinyllama-medtext-lora/chat_template.jinja',
 'tinyllama-medtext-lora/tokenizer.model',
 'tinyllama-medtext-lora/added_tokens.json',
 'tinyllama-medtext-lora/tokenizer.json')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

base_model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

adapter_path = "/content/tinyllama-medtext-lora"
model = PeftModel.from_pretrained(base_model, adapter_path)

model = model.merge_and_unload()

output_path = "/content/tinyllama-custom"
model.save_pretrained(output_path)

tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.save_pretrained(output_path)

print(f"Merged model saved to: {output_path}")

✅ Merged model saved to: /content/tinyllama-custom


In [ ]:
import shutil
shutil.make_archive("tinyllama-custom", "zip", "tinyllama-custom")
from google.colab import files
files.download("tinyllama-custom.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>